### Development of script to measure change in bikeability curve vars


To do:

- Pull through bikeability code and review carefully for accuracy / efficiency
- Write up what the process is for generating pareto curve (with anaysis of efficiency)
- Write up how feature are derived from curve and what each means in practical terms
- Run through process with example LSOAs and analyse outputs
- What are the correct implementation details (like parameter weightings)
- What parts of code are inefficient and should be run offline and which can be optimised more dynamically?
- Automate process to run for a list of LSOA

In [1]:
from helper_functions import expand_bbox, create_bounding_box, create_graph, get_edges_from_geom, coarsen_network, assign_weights, compute_pareto_fronts, dict_to_array, generate_reference_point, hypervolume, spread_diversity, compute_signed_area, compute_trade_off_rate, haversine_distance
import psycopg2
import pyproj
from dotenv import load_dotenv
import os
import geopandas as gpd
import pandas as pd
import random
from shapely.geometry import box

In [2]:
# Get DB username and password from local .env file
load_dotenv()
db_user = os.getenv('DB_USER')
db_password = os.getenv('DB_PASSWORD')

#Connect to DB
con = psycopg2.connect(database="ukrn", user=db_user, password=db_password, host="/var/run/postgresql/")
project = pyproj.Transformer.from_proj(pyproj.Proj(init='epsg:4326'),pyproj.Proj(init='epsg:3857'))
project_reverse = pyproj.Transformer.from_proj(pyproj.Proj(init='epsg:3857'),pyproj.Proj(init='epsg:4326'))

/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferre

In [3]:
# Import MSOA lookup
msoas = gpd.read_file('data/MSOA_EngWal_Dec_2011_Generalised_ClippedEW_0/Middle_Layer_Super_Output_Areas_December_2011_Generalised_Clipped_Boundaries_in_England_and_Wales.shp').to_crs(4326).set_index('msoa11cd')

# Import MSOA 2011 OD data
od_data = pd.read_parquet('data/od_2011.parquet')

#Import LSOAs
lsoas = gpd.read_file('data/LSOA_2011_Boundaries_Super_Generalised_Clipped_BSC_EW_V4_6029841263726194941.gpkg').to_crs(4326)
lsoas = pd.concat([lsoas, lsoas.bounds], axis=1)

#Import lsoa to msoa look up
lookup = pd.read_csv('data/PCD11_OA11_LSOA11_MSOA11_LAD11_EW_LU_aligned_v2.csv')
lsoas_ids = list(lsoas['LSOA11CD'])
lsoas = lsoas.set_index('LSOA11CD')

#Get OA Boundaries
oas = gpd.read_file('data/Output_Areas_Dec_2011_Boundaries_EW_BGC_2022_3211360474256931029.geojson').to_crs(4326)
oa_centroids = []
for i,r in oas.iterrows():
    oa_centroids.append(r['geometry'].centroid)
oas['centroid'] = oa_centroids
oa_centroids = gpd.GeoDataFrame(oas[['OA11CD','centroid']], geometry='centroid')
oa_centroids = oa_centroids.set_index('OA11CD')
oas = oas.set_index('OA11CD')

/tmp/ipykernel_548484/2119830582.py:12: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  lookup = pd.read_csv('data/PCD11_OA11_LSOA11_MSOA11_LAD11_EW_LU_aligned_v2.csv')


In [4]:
bbx_expansion = 0.5

In [91]:
lsoa_id = random.sample(lsoas_ids,1)[0]

lsoa_info = lsoas.loc[lsoa_id]
lsoa_lookup = lookup[lookup['LSOA11CD'] == lsoa_id][:1]

features_append = {}
features_append['lsoa'] = lsoa_id

expanded_bbox, expanded_geometry = expand_bbox((lsoa_info['minx'], lsoa_info['miny'], lsoa_info['maxx'], lsoa_info['maxy']), expansion_distance_km=bbx_expansion)

#OD Cycle to Work Network
bike_ods = od_data[(od_data['geo_code1'] == lsoa_lookup['MSOA11CD'].values[0]) & (od_data['bicycle'] > 0)][['geo_code2','bicycle']].set_index('geo_code2')
bike_ods['geometry'] = msoas['geometry']
bike_ods = bike_ods.dropna()
origin_geom = lsoa_info['geometry']
destination_geom = msoas.loc[bike_ods['bicycle'].idxmax()]['geometry']
bounding_box_coords = create_bounding_box(expanded_geometry, destination_geom)
od_bbox = box(bounding_box_coords[0],bounding_box_coords[1],bounding_box_coords[2],bounding_box_coords[3])

#Get 2016 networks
sql = "select r.* from import.rn2016_roads as r join lsoas.lsoas2011 as s ON ST_Intersects(r.geometry, s.shape) and s.lsoa11cd = '{}';".format(lsoa_id)
lsoa_edges_2016 = gpd.read_postgis(sql, con, geom_col = 'geometry').to_crs(4326)
G_lsoa_2016, nodes_lsoa_2016, edges_lsoa_2016 = create_graph(lsoa_edges_2016)

expanded_edges_2016 = get_edges_from_geom(expanded_geometry, con, project, 'rn2016_roads')
G_expanded_2016, nodes_expanded_2016, edges_expanded_2016 = create_graph(expanded_edges_2016)

od_edges_2016 = get_edges_from_geom(od_bbox, con, project, 'rn2016_roads')
G_od_2016, nodes_od_2016, edges_od_2016 = create_graph(od_edges_2016)

#Get 2021 networks
sql = "select r.* from import.rn2021_roads as r join lsoas.lsoas2011 as s ON ST_Intersects(r.geometry, s.shape) and s.lsoa11cd = '{}';".format(lsoa_id)
lsoa_edges_2021 = gpd.read_postgis(sql, con, geom_col = 'geometry').to_crs(4326)
G_lsoa_2021, nodes_lsoa_2021, edges_lsoa_2021 = create_graph(lsoa_edges_2021)

expanded_edges_2021 = get_edges_from_geom(expanded_geometry, con, project, 'rn2021_roads')
G_expanded_2021, nodes_expanded_2021, edges_expanded_2021 = create_graph(expanded_edges_2021)

od_edges_2021 = get_edges_from_geom(od_bbox, con, project, 'rn2021_roads')
G_od_2021, nodes_od_2021, edges_od_2021 = create_graph(od_edges_2021)

/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)
/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/shapely/ops.py:276: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  shell = type(geom.exterior)(zip(*func(*zip(*geom.exterior.coords))))
/home/chris/miniconda3/envs/ate/lib/python3.10/site-packages/pyproj/crs/crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/st

In [56]:
# Coarsen Network
G_c_21,oas_in_area  = coarsen_network(od_bbox,G_od_2021,nodes_od_2021,oas,oa_centroids)
G_c_16,oas_in_area  = coarsen_network(od_bbox,G_od_2016,nodes_od_2016,oas,oa_centroids)

In [7]:
import pyproj
import geopandas as gpd
import pandas as pd
from shapely.geometry import box, Point
from shapely import wkt
from functools import partial
from shapely.ops import transform
from shapely.ops import split
import networkx as nx
import heapq
from sortedcontainers import SortedDict
import math
import numpy as np
from scipy.spatial import ConvexHull
import ast
from shapely import to_geojson
from geojson_length import calculate_distance, Unit

def sample_dataframe(df, min_samples ,rate=0.1):
    """
    Samples rows from the input DataFrame based on a variable sampling rate.
    
    Parameters:
    df (pd.DataFrame): The input DataFrame.
    rate (float): The sampling rate for DataFrames with more than 10 rows.
    
    Returns:
    pd.DataFrame: The sampled DataFrame.
    """
    num_rows = len(df)
    
    if num_rows <= min_samples:
        # If the number of rows is 10 or less, return the entire DataFrame
        return df
    else:
        # Calculate the sample size
        sample_size = max(min_samples, int(num_rows * rate))
        
        # Sample the DataFrame
        sampled_df = df.sample(n=sample_size, random_state=1)
        return sampled_df

def mean_of_list(lst):
    return sum(lst)/len(lst)

### Try with LSOAs

In [92]:
lsoa_centroids = []
for i,r in lsoas.iterrows():
    lsoa_centroids.append(r['geometry'].centroid)
lsoas['centroid'] = lsoa_centroids
lsoa_centroids = gpd.GeoDataFrame(lsoas[['centroid']], geometry='centroid')

In [93]:
base_graph_nodes = nodes_od_2021
base_graph = G_od_2021

od_polygon = gpd.GeoSeries([od_bbox])
oas_in_area = lsoa_centroids[lsoa_centroids.intersects(od_polygon.unary_union)]

In [94]:
# Network nodes as points
network_node_points_list = []
for i,r in base_graph_nodes.iterrows():
    network_node_points_list.append(Point([r['latitude'],r['longitude']]))

base_graph_nodes['geometry'] = network_node_points_list
base_graph_nodes = gpd.GeoDataFrame(base_graph_nodes, geometry = 'geometry')

In [95]:
area_to_nodes = {}
num_nodes_per_oa = []
for oa in list(oas_in_area.index):
    oa_geom = gpd.GeoSeries([lsoas.loc[oa]['geometry']])
    oa_nodes = base_graph_nodes[base_graph_nodes.intersects(oa_geom.unary_union)]
    area_to_nodes[oa] = oa_nodes
    num_nodes_per_oa.append(len(oa_nodes))

In [96]:
#LTS1
lts_1_edges = [(u, v, d) for u, v, d in base_graph.edges(data=True) if d.get('lts', 0) <= 1]
G_lts1 = nx.Graph()
G_lts1.add_nodes_from(list(base_graph.nodes))
G_lts1.add_edges_from(lts_1_edges)

#LTS2
lts_2_edges = [(u, v, d) for u, v, d in base_graph.edges(data=True) if d.get('lts', 0) <= 2]
G_lts2 = nx.Graph()
G_lts2.add_nodes_from(list(base_graph.nodes))
G_lts2.add_edges_from(lts_2_edges)

#LTS3
lts_3_edges = [(u, v, d) for u, v, d in base_graph.edges(data=True) if d.get('lts', 0) <= 3]
G_lts3 = nx.Graph()
G_lts3.add_nodes_from(list(base_graph.nodes))
G_lts3.add_edges_from(lts_3_edges)

#LTS4
lts_4_edges = [(u, v, d) for u, v, d in base_graph.edges(data=True) if d.get('lts', 0) <= 4]
G_lts4 = nx.Graph()
G_lts4.add_nodes_from(list(base_graph.nodes))
G_lts4.add_edges_from(lts_4_edges)

oas_in_area_list = list(oas_in_area.index)

print('Number of OAs : {}'.format(len(oas_in_area_list)))

Number of OAs : 619


In [97]:
paths_lts_1 = {}
paths_lts_2 = {}
paths_lts_3 = {}
paths_lts_4 = {}
operations = 0

for o in oas_in_area_list:
    paths_lts_1[o] = {}
    paths_lts_2[o] = {}
    paths_lts_3[o] = {}
    paths_lts_4[o] = {}
    for d in oas_in_area_list:
        operations += 1
        paths_lts_1[o][d] = []
        paths_lts_2[o][d] = []
        paths_lts_3[o][d] = []
        paths_lts_4[o][d] = []
        
print(operations)

383161


In [98]:
# Define the original layers as MultiGraphs

operations = 0

for o in oas_in_area_list:
    origin_nodes = sample_dataframe(area_to_nodes[o],4,0.1)
    for next_o_node in list(origin_nodes['node']):

        all_sp_lts1 = nx.single_source_dijkstra_path_length(G_lts1,next_o_node, weight = 'length')
        operations += 1
        all_sp_lts2 = nx.single_source_dijkstra_path_length(G_lts2,next_o_node, weight = 'length')
        operations += 1
        all_sp_lts3 = nx.single_source_dijkstra_path_length(G_lts3,next_o_node, weight = 'length')
        operations += 1
        all_sp_lts4 = nx.single_source_dijkstra_path_length(G_lts4,next_o_node, weight = 'length')
        operations += 1

        for d in oas_in_area_list:
            
            operations += 1
            
            destination_nodes = list(area_to_nodes[d]['node'])

            #Add LTS 1 Routes to Dict
            paths_to_dest = {key: all_sp_lts1[key] for key in destination_nodes if key in all_sp_lts1}
            #Remove 0 elements
            paths_to_dest = [element for element in list(paths_to_dest.values()) if element != 0]
            #Add found routes to dictionary
            paths_lts_1[o][d].extend(paths_to_dest)

            #Add LTS 2 Routes to Dict
            paths_to_dest = {key: all_sp_lts2[key] for key in destination_nodes if key in all_sp_lts2}
            #Remove 0 elements
            paths_to_dest = [element for element in list(paths_to_dest.values()) if element != 0]
            #Add found routes to dictionary
            paths_lts_2[o][d].extend(paths_to_dest)
            
            #Add LTS 3 Routes to Dict
            paths_to_dest = {key: all_sp_lts3[key] for key in destination_nodes if key in all_sp_lts3}
            #Remove 0 elements
            paths_to_dest = [element for element in list(paths_to_dest.values()) if element != 0]
            #Add found routes to dictionary
            paths_lts_3[o][d].extend(paths_to_dest)
            
            #Add LTS 4 Routes to Dict
            paths_to_dest = {key: all_sp_lts4[key] for key in destination_nodes if key in all_sp_lts4}
            #Remove 0 elements
            paths_to_dest = [element for element in list(paths_to_dest.values()) if element != 0]
            #Add found routes to dictionary
            paths_lts_4[o][d].extend(paths_to_dest)
            
print(operations)

1825390


In [89]:
G_c = nx.DiGraph()
G_c.add_nodes_from(oas_in_area_list)

for o in oas_in_area_list:
    for d in oas_in_area_list:
        distances = []
        if len(paths_lts_1[o][d]) > 0:
            distances.append(mean_of_list(paths_lts_1[o][d]))
        else:
            distances.append(0)

        if len(paths_lts_2[o][d]) > 0:
            distances.append(mean_of_list(paths_lts_2[o][d]))
        else:
            distances.append(0)

        if len(paths_lts_3[o][d]) > 0:
            distances.append(mean_of_list(paths_lts_3[o][d]))
        else:
            distances.append(0)
            
        if len(paths_lts_4[o][d]) > 0:
            distances.append(mean_of_list(paths_lts_4[o][d]))
        else:
            distances.append(0)
        
        if sum(distances) > 0:
            G_c.add_edge(o, d, distance=distances)

In [14]:
# Get Bikability Curve
G_c_21=assign_weights(G_c_21,'Linear')
G_c_16=assign_weights(G_c_16,'Linear')

In [15]:
# Get OAs of origin
origin_lsoa = lsoa_info.name
origin_oas = list(set(list(lookup[lookup['LSOA11CD'] == origin_lsoa]['OA11CD'])))

# Get LSOAs in MSOA of destination
destination_msoa = list(set(list(lookup[lookup['MSOA11CD'] == msoas.loc[bike_ods['bicycle'].idxmax()].name]['OA11CD'])))

In [17]:
metrics_list = []
i = 0

for next_o in origin_oas:
    
    sp = compute_pareto_fronts(G_c_21,next_o)
    
    for next_d in destination_msoa:
        if sp.get(next_d) != None:
            length_risk_curve = dict_to_array(sp.get(next_d))
            if len(length_risk_curve) > 2:
                reference_point = generate_reference_point(length_risk_curve)
                hv = hypervolume(length_risk_curve, reference_point)
                diversity = spread_diversity(length_risk_curve)
                signed_area, hull = compute_signed_area(length_risk_curve)
                avg_trade_off_rate, trade_off_rates = compute_trade_off_rate(length_risk_curve)
                num_routes = len(length_risk_curve)
                distance = haversine_distance(oas.loc[next_o]['centroid'], oas.loc[next_d]['centroid'])
                
                metrics_list.append([hv, diversity, signed_area, avg_trade_off_rate, num_routes, distance])
        
        i += 1

In [ ]:

# Get OD pareto curves



metrics_list = []
i = 0

for next_o in origin_oas:
    
    sp = compute_pareto_fronts(G_c,next_o)
    
    for next_d in destination_msoa:
        if sp.get(next_d) != None:
            length_risk_curve = dict_to_array(sp.get(next_d))
            if len(length_risk_curve) > 2:
                reference_point = generate_reference_point(length_risk_curve)
                hv = hypervolume(length_risk_curve, reference_point)
                diversity = spread_diversity(length_risk_curve)
                signed_area, hull = compute_signed_area(length_risk_curve)
                avg_trade_off_rate, trade_off_rates = compute_trade_off_rate(length_risk_curve)
                num_routes = len(length_risk_curve)
                distance = haversine_distance(oas.loc[next_o]['centroid'], oas.loc[next_d]['centroid'])
                
                metrics_list.append([hv, diversity, signed_area, avg_trade_off_rate, num_routes, distance])
        
        i += 1

In [ ]:
all_vars = {}

# Select a random 100 LSOAs
sample_lsoas = random.sample(lsoas_ids, 500)

progress = 0

for lsoa_id in sample_lsoas:
    try:
        progress += 1
        print('Progress : {}'.format(progress))
        
        all_vars[lsoa_id] = {}

        #Construct Networks

        lsoa_info = lsoas.loc[lsoa_id]
        lsoa_lookup = lookup[lookup['LSOA11CD'] == lsoa_id][:1]

        features_append = {}
        features_append['lsoa'] = lsoa_id

        expanded_bbox, expanded_geometry = expand_bbox((lsoa_info['minx'], lsoa_info['miny'], lsoa_info['maxx'], lsoa_info['maxy']), expansion_distance_km=bbx_expansion)

        #OD Cycle to Work Network
        bike_ods = od_data[(od_data['geo_code1'] == lsoa_lookup['MSOA11CD'].values[0]) & (od_data['bicycle'] > 0)][['geo_code2','bicycle']].set_index('geo_code2')
        bike_ods['geometry'] = msoas['geometry']
        bike_ods = bike_ods.dropna()
        origin_geom = lsoa_info['geometry']
        destination_geom = msoas.loc[bike_ods['bicycle'].idxmax()]['geometry']
        bounding_box_coords = create_bounding_box(expanded_geometry, destination_geom)
        od_bbox = box(bounding_box_coords[0],bounding_box_coords[1],bounding_box_coords[2],bounding_box_coords[3])

        #Get 2016 networks
        sql = "select r.* from import.rn2016_roads as r join lsoas.lsoas2011 as s ON ST_Intersects(r.geometry, s.shape) and s.lsoa11cd = '{}';".format(lsoa_id)
        lsoa_edges_2016 = gpd.read_postgis(sql, con, geom_col = 'geometry').to_crs(4326)
        G_lsoa_2016, nodes_lsoa_2016, edges_lsoa_2016 = create_graph(lsoa_edges_2016)

        expanded_edges_2016 = get_edges_from_geom(expanded_geometry, con, project, 'rn2016_roads')
        G_expanded_2016, nodes_expanded_2016, edges_expanded_2016 = create_graph(expanded_edges_2016)

        od_edges_2016 = get_edges_from_geom(od_bbox, con, project, 'rn2016_roads')
        G_od_2016, nodes_od_2016, edges_od_2016 = create_graph(od_edges_2016)

        #Get 2021 networks
        sql = "select r.* from import.rn2021_roads as r join lsoas.lsoas2011 as s ON ST_Intersects(r.geometry, s.shape) and s.lsoa11cd = '{}';".format(lsoa_id)
        lsoa_edges_2021 = gpd.read_postgis(sql, con, geom_col = 'geometry').to_crs(4326)
        G_lsoa_2021, nodes_lsoa_2021, edges_lsoa_2021 = create_graph(lsoa_edges_2021)

        expanded_edges_2021 = get_edges_from_geom(expanded_geometry, con, project, 'rn2021_roads')
        G_expanded_2021, nodes_expanded_2021, edges_expanded_2021 = create_graph(expanded_edges_2021)

        od_edges_2021 = get_edges_from_geom(od_bbox, con, project, 'rn2021_roads')
        G_od_2021, nodes_od_2021, edges_od_2021 = create_graph(od_edges_2021)

        # Coarsen Network
        G_c,oas_in_area  = coarsen_network(od_bbox,G_od_2021,nodes_od_2021,oas)

        # Get Bikability Curve
        G_c=assign_weights(G_c,'Linear')

        # Get OD pareto curves

        # Get OAs of origin
        origin_lsoa = lsoa_info.name
        origin_oas = list(set(list(lookup[lookup['LSOA11CD'] == origin_lsoa]['OA11CD'])))

        # Get LSOAs in MSOA of destination
        destination_msoa = list(set(list(lookup[lookup['MSOA11CD'] == msoas.loc[bike_ods['bicycle'].idxmax()].name]['OA11CD'])))

        metrics_list = []
        i = 0

        for next_o in origin_oas:
            
            sp = compute_pareto_fronts(G_c,next_o)
            
            for next_d in destination_msoa:
                if sp.get(next_d) != None:
                    length_risk_curve = dict_to_array(sp.get(next_d))
                    if len(length_risk_curve) > 2:
                        reference_point = generate_reference_point(length_risk_curve)
                        hv = hypervolume(length_risk_curve, reference_point)
                        diversity = spread_diversity(length_risk_curve)
                        signed_area, hull = compute_signed_area(length_risk_curve)
                        avg_trade_off_rate, trade_off_rates = compute_trade_off_rate(length_risk_curve)
                        num_routes = len(length_risk_curve)
                        distance = haversine_distance(oas.loc[next_o]['centroid'], oas.loc[next_d]['centroid'])
                        
                        metrics_list.append([hv, diversity, signed_area, avg_trade_off_rate, num_routes, distance])
                
                i += 1
                
        metrics = np.array(metrics_list)

        all_vars[lsoa_id]['bike curve'] = metrics
        all_vars[lsoa_id]['change vars'] = model_vars[model_vars['LSOA_code'] == lsoa_id][['cycling_percentage_point_difference','walking_percentage_point_difference']].values
        with open('data/bike_curve_outputs_2.pkl', 'wb') as f:
            pickle.dump(all_vars, f)
    except:
        pass